In [18]:
# Lib imports
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import numpy as np

In [19]:
# DATASET DIRECTORY CONFIGURATION
# Download and unzip the dataset from Kaggle, set the directory paths accordingly.
train_dir = "archive/train"
test_dir = "archive/test"

In [20]:
# IMAGE PARAMETERS
# Used to resize the input images, also will determine the input size of your input layer.
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

In [ ]:
# DATA PREPROCESSING & AUGMENTATION

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 3788 images belonging to 2 classes.
Found 945 images belonging to 2 classes.
Found 1184 images belonging to 2 classes.


In [34]:
# IMPROVED CNN MODEL ARCHITECTURE
from tensorflow.keras import regularizers

# 1. OPTIMIZER CONFIGURATION
initial_learning_rate = 0.001

# Adjusted decay steps to match dataset size better (previously 10000 was too high to trigger often)
# Assuming roughly 119 steps per epoch, 1000 steps = decay every ~8 epochs
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=1000,
    decay_rate=0.9,
    staircase=True
)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# 2. MODEL DEFINITION WITH REGULARIZATION & DROPOUT
model = models.Sequential([
    # Block 1
    layers.Conv2D(32, (3, 3), padding='same', activation='relu',
    kernel_regularizer=regularizers.l2(0.001),
    input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),

    # Block 2
    layers.Conv2D(64, (3, 3), padding='same', activation='relu',
    kernel_regularizer=regularizers.l2(0.001)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.35),

    # Block 3
    layers.Conv2D(128, (3, 3), padding='same', activation='relu',
    kernel_regularizer=regularizers.l2(0.001)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.45),

    # Classification Head
    layers.Flatten(),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

# Print summary to verify the changes
model.summary()

c:\Users\Karlo Roel\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 128, 128, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 64, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 32, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 32768)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │     4,194,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,287,809 (16.36 MB)

 Trainable params: 4,287,809 (16.36 MB)

 Non-trainable params: 0 (0.00 B)

In [36]:
# Configure the model optimizers, loss function, and metrics
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) # old
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [37]:
# TRAINING THE CNN
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

Epoch 1/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 300s 2s/step - accuracy: 0.6573 - loss: 0.9263 - val_accuracy: 0.6942 - val_loss: 0.7383
Epoch 2/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 200s 2s/step - accuracy: 0.7680 - loss: 0.6470 - val_accuracy: 0.8233 - val_loss: 0.5627
Epoch 3/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 201s 2s/step - accuracy: 0.7825 - loss: 0.5787 - val_accuracy: 0.8201 - val_loss: 0.5140
Epoch 4/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 116s 969ms/step - accuracy: 0.8068 - loss: 0.5180 - val_accuracy: 0.8360 - val_loss: 0.4529
Epoch 5/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 103s 856ms/step - accuracy: 0.8271 - loss: 0.4849 - val_accuracy: 0.8550 - val_loss: 0.4317
Epoch 6/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 103s 862ms/step - accuracy: 0.8376 - loss: 0.4661 - val_accuracy: 0.8402 - val_loss: 0.4350
Epoch 7/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 152s 1s/step - accuracy: 0.8366 - loss: 0.4482 - val_accuracy: 0.8444 - val_loss: 0.4599
Epoch 8/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 140s 1s/step - accuracy: 0.8487 - loss: 0.4329 -

In [38]:
# EVALUATE THE MODEL
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")

37/37 ━━━━━━━━━━━━━━━━━━━━ 32s 872ms/step - accuracy: 0.8606 - loss: 0.3821
Test Accuracy: 0.8606418967247009


In [63]:
# SAVE THE MODEL
model.save('exercise_6_trained_model_improved.h5')

In [ ]:
# SIMPLE INFERENCE SCRIPT
from tensorflow.keras.preprocessing import image

def predict_image(img_path, model_path='exercise_6_trained_model_improved.h5'):
    # Load model for inference without compiling so compiled metrics warning is avoided
    model = tf.keras.models.load_model(model_path, compile=False)
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = model.predict(img_array)[0,0]
    label = "Chihuahua" if pred >= 0.5 else "Muffin"
    print(f"Prediction: {label} (confidence: {pred:.2f})")

In [24]:
predict_image("chocolate-chip-muffins-featured.jpg")
predict_image("Chihuahua-standing-in-three-quarter-view.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
Prediction: Chihuahua (confidence: 0.96)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
Prediction: Muffin (confidence: 0.05)


#3 HORSES AND HUMANS

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras import layers, models, regularizers


In [ ]:
# Check current working directory
current_dir = os.getcwd()
print(f"Current Notebook Directory: {current_dir}")

# Correct dataset structure:
# dataset/horse-or-human/train
# dataset/horse-or-human/validation

train_dir = os.path.join(current_dir, "dataset", "horse-or-human", "train")
validation_dir = os.path.join(current_dir, "dataset", "horse-or-human", "validation")

# Verification
if not os.path.exists(train_dir):
    raise FileNotFoundError(f"❌ TRAIN FOLDER NOT FOUND:\n{train_dir}")
else:
    print(f"✅ Training folder found: {train_dir}")

if not os.path.exists(validation_dir):
    raise FileNotFoundError(f"❌ VALIDATION FOLDER NOT FOUND:\n{validation_dir}")
else:
    print(f"✅ Validation folder found: {validation_dir}")

Current Notebook Directory: c:\Users\Karlo Roel\Downloads\25-26\CS 3A\KARLO_MONTENEGRO
✅ Training folder found: c:\Users\Karlo Roel\Downloads\25-26\CS 3A\KARLO_MONTENEGRO\dataset\horse-or-human\train
✅ Validation folder found: c:\Users\Karlo Roel\Downloads\25-26\CS 3A\KARLO_MONTENEGRO\dataset\horse-or-human\validation


In [ ]:
#DATA AUGMENTATION & LOADERS
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

print("\nLoading images...")

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

val_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

# mapping example: {0: 'horses', 1: 'humans'}
class_names = {v: k for k, v in train_generator.class_indices.items()}
print("\nClass mapping:", class_names)



Loading images...
Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.

Class mapping: {0: 'horses', 1: 'humans'}


In [ ]:
#CNN MODEL (Dropout + L2 Regularization)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0003)
l2_reg = regularizers.l2(0.0003)

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', padding='same',
                kernel_regularizer=l2_reg,
                input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.2),

    layers.Conv2D(64, (3,3), activation='relu', padding='same',
                kernel_regularizer=l2_reg),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.25),

    layers.Conv2D(128, (3,3), activation='relu', padding='same',
                kernel_regularizer=l2_reg),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.3),

    layers.Conv2D(256, (3,3), activation='relu', padding='same',
                kernel_regularizer=l2_reg),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.4),

    layers.Flatten(),
    layers.Dense(512, activation='relu', kernel_regularizer=l2_reg),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

model.summary()

model.compile(
    optimizer=optimizer,
    loss="binary_crossentropy",
    metrics=["accuracy"]
)


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_23 (Conv2D)              │ (None, 128, 128, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_23 (MaxPooling2D) │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_22 (Dropout)            │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_24 (Conv2D)              │ (None, 64, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_24 (MaxPooling2D) │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_23 (Dropout)            │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_25 (Conv2D)              │ (None, 32, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_25 (MaxPooling2D) │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_24 (Dropout)            │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_26 (Conv2D)              │ (None, 16, 16, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_26 (MaxPooling2D) │ (None, 8, 8, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_25 (Dropout)            │ (None, 8, 8, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_7 (Flatten)             │ (None, 16384)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 512)            │     8,389,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_26 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,778,049 (33.49 MB)

 Trainable params: 8,778,049 (33.49 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# CORRECTED PATHS — FIX THIS PART ONLY
train_dir = "dataset/horse-or-human/train"
validation_dir = "dataset/horse-or-human/validation"

# PREPROCESSING & AUGMENTATION
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

# Augmentation for Training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Only Rescaling for Validation
validation_datagen = ImageDataGenerator(rescale=1./255)

print("\nLoading images...")
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

val_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

# Save class names
class_names = {v: k for k, v in train_generator.class_indices.items()}
print(f"Classes found: {class_names}")



Loading images...
Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.
Classes found: {0: 'horses', 1: 'humans'}


In [ ]:
#TRAINING
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=15
)


Epoch 1/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - accuracy: 0.5005 - loss: 1.1554 - val_accuracy: 0.5000 - val_loss: 1.0372
Epoch 2/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - accuracy: 0.5297 - loss: 1.0079 - val_accuracy: 0.3789 - val_loss: 1.0039
Epoch 3/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 32s 963ms/step - accuracy: 0.6602 - loss: 0.9165 - val_accuracy: 0.5000 - val_loss: 1.0472
Epoch 4/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 38s 1s/step - accuracy: 0.6923 - loss: 0.8520 - val_accuracy: 0.5000 - val_loss: 1.0175
Epoch 5/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 38s 1s/step - accuracy: 0.7118 - loss: 0.8107 - val_accuracy: 0.5000 - val_loss: 1.1493
Epoch 6/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 30s 898ms/step - accuracy: 0.7332 - loss: 0.7996 - val_accuracy: 0.5000 - val_loss: 1.0488
Epoch 7/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 31s 929ms/step - accuracy: 0.7692 - loss: 0.7198 - val_accuracy: 0.5391 - val_loss: 1.1880
Epoch 8/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 27s 824ms/step - accuracy: 0.7751 - loss: 0.6654 - val_accuracy: 0.5117

In [ ]:
#EVALUATION
val_loss, val_acc = model.evaluate(val_generator)
print(f"\nValidation Accuracy: {val_acc*100:.2f}%")


8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 432ms/step - accuracy: 0.5117 - loss: 5.8941

Validation Accuracy: 51.17%


In [ ]:
#SAVE MODEL
MODEL_PATH = "exercise_6_custom_montenegro.h5"
model.save(MODEL_PATH)
print(f"Model saved as: {MODEL_PATH}")


Model saved as: exercise_6_custom_montenegro.h5


In [ ]:
#FIXED PREDICTION FUNCTION
def predict_image(img_path, model_path="exercise_6_custom_montenegro.h5"):
    try:
        # Load saved model
        loaded_model = tf.keras.models.load_model(model_path)

        # Load and preprocess image
        img = image.load_img(img_path, target_size=IMG_SIZE)
        img_array = image.img_to_array(img) / 255.0
        img_array = np.expand_dims(img_array, axis=0)

        # Predict
        score = loaded_model.predict(img_array)[0][0]

        if score > 0.5:
            label = "human"
            confidence = score
        else:
            label = "horse"
            confidence = 1 - score

        print(f"Prediction: {label.upper()}  |  Confidence: {confidence:.2f}")

    except Exception as e:
        print(f"Could not process image {img_path}: {e}")


In [ ]:
predict_image("Horses-Facts-Characteristics-Types-Behavior-Diet-More.jpg.webp")
predict_image("OIP (1).jpg")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 565ms/step
Prediction: HORSE  |  Confidence: 0.95


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 305ms/step
Prediction: HUMAN  |  Confidence: 1.00


#4 ResNet CNN Architecture for Chihuahua vs Muffin Classification

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# DATASET DIRECTORY CONFIGURATION
train_dir = r"D:\Downloads\BOLIVAR_MONTENEGRO_EXERCISE_6\train"
test_dir = r"D:\Downloads\BOLIVAR_MONTENEGRO_EXERCISE_6\test"

In [ ]:
# IMAGE PARAMETERS
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

In [ ]:
# DATA PREPROCESSING & AUGMENTATION
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

print(f"Found {train_generator.samples} training images")
print(f"Found {val_generator.samples} validation images")
print(f"Found {test_generator.samples} test images")

Found 3788 images belonging to 2 classes.


Found 945 images belonging to 2 classes.
Found 1184 images belonging to 2 classes.
Found 3788 training images
Found 945 validation images
Found 1184 test images
Found 1184 images belonging to 2 classes.
Found 3788 training images
Found 945 validation images
Found 1184 test images


In [ ]:
# RESIDUAL BLOCK DEFINITION
class ResidualBlock(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size=3, stride=1, activation='relu', **kwargs):
        super(ResidualBlock, self).__init__(**kwargs)
        self.filters = filters
        self.kernel_size = kernel_size
        self.stride = stride
        self.activation = activation
        
        # Main path
        self.conv1 = layers.Conv2D(filters, kernel_size, strides=stride, 
                                  padding='same', use_bias=False)
        self.bn1 = layers.BatchNormalization()
        self.act1 = layers.Activation(activation)
        
        self.conv2 = layers.Conv2D(filters, kernel_size, strides=1,
                                  padding='same', use_bias=False)
        self.bn2 = layers.BatchNormalization()
        
        # Shortcut path
        self.shortcut = layers.Conv2D(filters, 1, strides=stride, 
                                     padding='same', use_bias=False) if stride != 1 else layers.Identity()
        self.bn_shortcut = layers.BatchNormalization() if stride != 1 else layers.Identity()
        
        self.add = layers.Add()
        self.act2 = layers.Activation(activation)
        
    def call(self, inputs, training=False):
        # Main path
        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        x = self.act1(x)
        
        x = self.conv2(x)
        x = self.bn2(x, training=training)
        
        # Shortcut path
        shortcut = self.shortcut(inputs)
        if self.stride != 1:
            shortcut = self.bn_shortcut(shortcut, training=training)
        
        # Add and activate
        x = self.add([x, shortcut])
        x = self.act2(x)
        
        return x



In [ ]:
# RESNET-STYLE CNN ARCHITECTURE
def build_resnet_model(input_shape=(128, 128, 3)):
    inputs = tf.keras.Input(shape=input_shape)
    
    # Initial Convolution
    x = layers.Conv2D(32, 7, strides=2, padding='same', use_bias=False)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D(3, strides=2, padding='same')(x)
    
    # Residual Blocks
    # Block 1
    x = ResidualBlock(32, stride=1)(x)
    x = layers.Dropout(0.2)(x)
    
    # Block 2
    x = ResidualBlock(64, stride=2)(x)
    x = layers.Dropout(0.3)(x)
    
    # Block 3
    x = ResidualBlock(128, stride=2)(x)
    x = layers.Dropout(0.4)(x)
    
    # Block 4
    x = ResidualBlock(256, stride=2)(x)
    x = layers.Dropout(0.4)(x)
    
    # Global Average Pooling
    x = layers.GlobalAveragePooling2D()(x)
    
    # Fully Connected Layers
    x = layers.Dense(512, activation='relu', 
                    kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    
    x = layers.Dense(256, activation='relu', 
                    kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    
    # Output Layer
    outputs = layers.Dense(1, activation='sigmoid')(x)
    
    model = tf.keras.Model(inputs, outputs)
    return model

In [ ]:
# BUILD AND COMPILE THE MODEL
print("Building ResNet model...")
resnet_model = build_resnet_model(input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))

# OPTIMIZER WITH LEARNING RATE SCHEDULE
initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=1000,
    decay_rate=0.9,
    staircase=True
)

# Use a float learning rate so callbacks (e.g. ReduceLROnPlateau) can modify it at runtime.
optimizer = tf.keras.optimizers.Adam(
    learning_rate=initial_learning_rate,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07
)

# COMPILE MODEL
resnet_model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy', 'precision', 'recall']
)



Building ResNet model...


In [ ]:
# MODEL SUMMARY
resnet_model.summary()

# CALLBACKS
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=5,
        min_lr=1e-7
    ),
    tf.keras.callbacks.ModelCheckpoint(
        'best_resnet_model.h5',
        monitor='val_accuracy',
        save_best_only=True,
        mode='max'
    )
]

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 64, 64, 32)     │         4,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_14          │ (None, 64, 64, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_9 (Activation)       │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_4                │ (None, 32, 32, 32)     │        18,688 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_5                │ (None, 16, 16, 64)     │        58,112 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_6                │ (None, 8, 8, 128)      │       230,912 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_7                │ (None, 4, 4, 256)      │       920,576 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 256)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 512)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_26          │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_27          │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           25

 Total params: 1,499,361 (5.72 MB)

 Trainable params: 1,494,945 (5.70 MB)

 Non-trainable params: 4,416 (17.25 KB)

In [ ]:
# TRAINING THE RESNET MODEL
print("Training ResNet model...")
history = resnet_model.fit(
    train_generator,
    epochs=30,
    validation_data=val_generator,
    callbacks=callbacks,
    verbose=1
)



Training ResNet model...
Epoch 1/30
Epoch 1/30
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 364ms/step - accuracy: 0.6512 - loss: 1.4655 - precision: 0.6200 - recall: 0.6554

119/119 ━━━━━━━━━━━━━━━━━━━━ 62s 441ms/step - accuracy: 0.7117 - loss: 1.3186 - precision: 0.6824 - recall: 0.6966 - val_accuracy: 0.5407 - val_loss: 2.2089 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 0.0010
Epoch 2/30
Epoch 2/30
119/119 ━━━━━━━━━━━━━━━━━━━━ 49s 415ms/step - accuracy: 0.8089 - loss: 1.0466 - precision: 0.7974 - recall: 0.7828 - val_accuracy: 0.5407 - val_loss: 2.5352 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 0.0010
Epoch 3/30
119/119 ━━━━━━━━━━━━━━━━━━━━ 49s 415ms/step - accuracy: 0.8089 - loss: 1.0466 - precision: 0.7974 - recall: 0.7828 - val_accuracy: 0.5407 - val_loss: 2.5352 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 0.0010
Epoch 3/30
119/119 ━━━━━━━━━━━━━━━━━━━━ 51s 431ms/step - accuracy: 0.8271 - loss: 0.9375 - precision: 0.8227 - recall: 0.7948 - val_accuracy: 0.5407 - val_loss: 2.7915 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 0.0010
Epoch 4/30
119/119 ━

119/119 ━━━━━━━━━━━━━━━━━━━━ 52s 437ms/step - accuracy: 0.8421 - loss: 0.8352 - precision: 0.8363 - recall: 0.8161 - val_accuracy: 0.8381 - val_loss: 0.8721 - val_precision: 0.7435 - val_recall: 0.9885 - learning_rate: 0.0010
Epoch 5/30
Epoch 5/30
119/119 ━━━━━━━━━━━━━━━━━━━━ 50s 420ms/step - accuracy: 0.8582 - loss: 0.7610 - precision: 0.8507 - recall: 0.8385 - val_accuracy: 0.6032 - val_loss: 1.7698 - val_precision: 0.5366 - val_recall: 0.9977 - learning_rate: 0.0010
Epoch 6/30
119/119 ━━━━━━━━━━━━━━━━━━━━ 50s 420ms/step - accuracy: 0.8582 - loss: 0.7610 - precision: 0.8507 - recall: 0.8385 - val_accuracy: 0.6032 - val_loss: 1.7698 - val_precision: 0.5366 - val_recall: 0.9977 - learning_rate: 0.0010
Epoch 6/30
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 381ms/step - accuracy: 0.8706 - loss: 0.7063 - precision: 0.8669 - recall: 0.8497

119/119 ━━━━━━━━━━━━━━━━━━━━ 53s 449ms/step - accuracy: 0.8633 - loss: 0.7056 - precision: 0.8548 - recall: 0.8460 - val_accuracy: 0.9397 - val_loss: 0.5415 - val_precision: 0.9435 - val_recall: 0.9240 - learning_rate: 0.0010
Epoch 7/30
Epoch 7/30
119/119 ━━━━━━━━━━━━━━━━━━━━ 47s 398ms/step - accuracy: 0.8743 - loss: 0.6447 - precision: 0.8653 - recall: 0.8603 - val_accuracy: 0.7069 - val_loss: 1.0923 - val_precision: 0.6139 - val_recall: 0.9747 - learning_rate: 0.0010
Epoch 8/30
119/119 ━━━━━━━━━━━━━━━━━━━━ 47s 398ms/step - accuracy: 0.8743 - loss: 0.6447 - precision: 0.8653 - recall: 0.8603 - val_accuracy: 0.7069 - val_loss: 1.0923 - val_precision: 0.6139 - val_recall: 0.9747 - learning_rate: 0.0010
Epoch 8/30
119/119 ━━━━━━━━━━━━━━━━━━━━ 52s 440ms/step - accuracy: 0.8788 - loss: 0.5981 - precision: 0.8730 - recall: 0.8615 - val_accuracy: 0.7947 - val_loss: 0.7215 - val_precision: 0.8774 - val_recall: 0.6429 - learning_rate: 0.0010
Epoch 9/30
119/119 ━━━━━━━━━━━━━━━━━━━━ 52s 440ms/st

119/119 ━━━━━━━━━━━━━━━━━━━━ 47s 398ms/step - accuracy: 0.9131 - loss: 0.4005 - precision: 0.9119 - recall: 0.8977 - val_accuracy: 0.9503 - val_loss: 0.3084 - val_precision: 0.9596 - val_recall: 0.9309 - learning_rate: 2.0000e-04
Epoch 14/30
Epoch 14/30
119/119 ━━━━━━━━━━━━━━━━━━━━ 49s 408ms/step - accuracy: 0.9155 - loss: 0.3912 - precision: 0.9172 - recall: 0.8971 - val_accuracy: 0.9280 - val_loss: 0.3567 - val_precision: 0.8845 - val_recall: 0.9700 - learning_rate: 2.0000e-04
Epoch 15/30
119/119 ━━━━━━━━━━━━━━━━━━━━ 49s 408ms/step - accuracy: 0.9155 - loss: 0.3912 - precision: 0.9172 - recall: 0.8971 - val_accuracy: 0.9280 - val_loss: 0.3567 - val_precision: 0.8845 - val_recall: 0.9700 - learning_rate: 2.0000e-04
Epoch 15/30
119/119 ━━━━━━━━━━━━━━━━━━━━ 48s 399ms/step - accuracy: 0.9153 - loss: 0.3792 - precision: 0.9113 - recall: 0.9034 - val_accuracy: 0.9101 - val_loss: 0.4130 - val_precision: 0.8428 - val_recall: 0.9885 - learning_rate: 2.0000e-04
Epoch 16/30
119/119 ━━━━━━━━━━━━

119/119 ━━━━━━━━━━━━━━━━━━━━ 47s 395ms/step - accuracy: 0.9263 - loss: 0.3273 - precision: 0.9177 - recall: 0.9224 - val_accuracy: 0.9545 - val_loss: 0.2541 - val_precision: 0.9688 - val_recall: 0.9309 - learning_rate: 4.0000e-05
Epoch 20/30
Epoch 20/30
119/119 ━━━━━━━━━━━━━━━━━━━━ 46s 386ms/step - accuracy: 0.9316 - loss: 0.3180 - precision: 0.9308 - recall: 0.9195 - val_accuracy: 0.9397 - val_loss: 0.2985 - val_precision: 0.8968 - val_recall: 0.9816 - learning_rate: 4.0000e-05
Epoch 21/30
119/119 ━━━━━━━━━━━━━━━━━━━━ 46s 386ms/step - accuracy: 0.9316 - loss: 0.3180 - precision: 0.9308 - recall: 0.9195 - val_accuracy: 0.9397 - val_loss: 0.2985 - val_precision: 0.8968 - val_recall: 0.9816 - learning_rate: 4.0000e-05
Epoch 21/30
119/119 ━━━━━━━━━━━━━━━━━━━━ 47s 399ms/step - accuracy: 0.9266 - loss: 0.3302 - precision: 0.9211 - recall: 0.9190 - val_accuracy: 0.9386 - val_loss: 0.2779 - val_precision: 0.9017 - val_recall: 0.9724 - learning_rate: 4.0000e-05
Epoch 22/30
119/119 ━━━━━━━━━━━━

119/119 ━━━━━━━━━━━━━━━━━━━━ 46s 390ms/step - accuracy: 0.9324 - loss: 0.3163 - precision: 0.9329 - recall: 0.9190 - val_accuracy: 0.9619 - val_loss: 0.2457 - val_precision: 0.9650 - val_recall: 0.9516 - learning_rate: 4.0000e-05
Epoch 23/30
Epoch 23/30
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 324ms/step - accuracy: 0.9171 - loss: 0.3262 - precision: 0.9075 - recall: 0.9083

119/119 ━━━━━━━━━━━━━━━━━━━━ 47s 392ms/step - accuracy: 0.9282 - loss: 0.3108 - precision: 0.9267 - recall: 0.9161 - val_accuracy: 0.9640 - val_loss: 0.2379 - val_precision: 0.9587 - val_recall: 0.9631 - learning_rate: 4.0000e-05
Epoch 24/30
Epoch 24/30
119/119 ━━━━━━━━━━━━━━━━━━━━ 46s 385ms/step - accuracy: 0.9311 - loss: 0.3027 - precision: 0.9272 - recall: 0.9224 - val_accuracy: 0.9418 - val_loss: 0.2692 - val_precision: 0.9871 - val_recall: 0.8848 - learning_rate: 4.0000e-05
Epoch 25/30
119/119 ━━━━━━━━━━━━━━━━━━━━ 46s 385ms/step - accuracy: 0.9311 - loss: 0.3027 - precision: 0.9272 - recall: 0.9224 - val_accuracy: 0.9418 - val_loss: 0.2692 - val_precision: 0.9871 - val_recall: 0.8848 - learning_rate: 4.0000e-05
Epoch 25/30
119/119 ━━━━━━━━━━━━━━━━━━━━ 46s 386ms/step - accuracy: 0.9287 - loss: 0.3108 - precision: 0.9273 - recall: 0.9167 - val_accuracy: 0.9619 - val_loss: 0.2338 - val_precision: 0.9585 - val_recall: 0.9585 - learning_rate: 4.0000e-05
Epoch 26/30
119/119 ━━━━━━━━━━━━

119/119 ━━━━━━━━━━━━━━━━━━━━ 47s 392ms/step - accuracy: 0.9343 - loss: 0.3091 - precision: 0.9342 - recall: 0.9218 - val_accuracy: 0.9714 - val_loss: 0.2190 - val_precision: 0.9857 - val_recall: 0.9516 - learning_rate: 4.0000e-05
Epoch 27/30
Epoch 27/30
119/119 ━━━━━━━━━━━━━━━━━━━━ 47s 396ms/step - accuracy: 0.9319 - loss: 0.3027 - precision: 0.9303 - recall: 0.9207 - val_accuracy: 0.9524 - val_loss: 0.2505 - val_precision: 0.9732 - val_recall: 0.9217 - learning_rate: 4.0000e-05
Epoch 28/30
119/119 ━━━━━━━━━━━━━━━━━━━━ 47s 396ms/step - accuracy: 0.9319 - loss: 0.3027 - precision: 0.9303 - recall: 0.9207 - val_accuracy: 0.9524 - val_loss: 0.2505 - val_precision: 0.9732 - val_recall: 0.9217 - learning_rate: 4.0000e-05
Epoch 28/30
119/119 ━━━━━━━━━━━━━━━━━━━━ 46s 390ms/step - accuracy: 0.9337 - loss: 0.2972 - precision: 0.9311 - recall: 0.9241 - val_accuracy: 0.9439 - val_loss: 0.2775 - val_precision: 0.9897 - val_recall: 0.8871 - learning_rate: 4.0000e-05
Epoch 29/30
119/119 ━━━━━━━━━━━━

In [26]:
# EVALUATE THE MODEL
print("Evaluating model...")
test_loss, test_accuracy, test_precision, test_recall = resnet_model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall: {test_recall:.4f}")



Evaluating model...
37/37 ━━━━━━━━━━━━━━━━━━━━ 18s 483ms/step - accuracy: 0.9468 - loss: 0.2810 - precision: 0.9512 - recall: 0.9320
37/37 ━━━━━━━━━━━━━━━━━━━━ 18s 483ms/step - accuracy: 0.9468 - loss: 0.2810 - precision: 0.9512 - recall: 0.9320
Test Accuracy: 0.9468
Test Precision: 0.9512
Test Recall: 0.9320
Test Accuracy: 0.9468
Test Precision: 0.9512
Test Recall: 0.9320


In [27]:
# SAVE THE FINAL MODEL
resnet_model.save('exercise_6_resnet_model.h5')
print("ResNet model saved as 'exercise_6_resnet_model.h5'")


ResNet model saved as 'exercise_6_resnet_model.h5'


In [28]:
# PREDICTION FUNCTION FOR RESNET MODEL
def predict_image_resnet(img_path, model_path='exercise_6_resnet_model.h5'):
    """Predict using the ResNet model"""
    try:
        # Load ResNet model for inference without compiling to suppress the warning
        model = tf.keras.models.load_model(
            model_path, 
            custom_objects={'ResidualBlock': ResidualBlock},
            compile=False
        )
        
        img = tf.keras.preprocessing.image.load_img(img_path, target_size=IMG_SIZE)
        img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
        img_array = np.expand_dims(img_array, axis=0)
        
        prediction = model.predict(img_array)[0][0]
        
        if prediction >= 0.5:
            label = "Chihuahua"
            confidence = prediction
        else:
            label = "Muffin"
            confidence = 1 - prediction
            
        print(f"ResNet Prediction: {label}")
        print(f"Confidence: {confidence:.4f}")
        print(f"Raw score: {prediction:.4f}")
        
        return label, confidence
        
    except Exception as e:
        print(f"Error during prediction: {e}")
        return None, None



In [29]:
# TEST PREDICTIONS
print("\nTesting ResNet model predictions:")
predict_image_resnet(r"D:\Downloads\BOLIVAR_MONTENEGRO_EXERCISE_6\train\muffin\img_0_46.jpg")
predict_image_resnet(r"D:\Downloads\BOLIVAR_MONTENEGRO_EXERCISE_6\train\chihuahua\img_0_19.jpg")


Testing ResNet model predictions:


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 378ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 378ms/step
ResNet Prediction: Chihuahua
Confidence: 0.9986
Raw score: 0.9986
ResNet Prediction: Chihuahua
Confidence: 0.9986
Raw score: 0.9986


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 354ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 354ms/step
ResNet Prediction: Muffin
Confidence: 0.9917
Raw score: 0.0083
ResNet Prediction: Muffin
Confidence: 0.9917
Raw score: 0.0083


('Muffin', np.float32(0.9916986))